In [1]:
import mysql.connector
from mysql.connector import Error
import pandas as pd

In [2]:
import tkinter as tk
from tkinter import messagebox
from tkcalendar import DateEntry
from tkinter import ttk
from datetime import datetime

In [3]:
class ConexionDB:

    #Conexion requisitos 
    def __init__(self, host, user, password, database):
        self.host=host
        self.user=user
        self.password=password
        self.database=database
        self.connection=None
        self.cursor=None
    
    #Funcion conexion

    
    def conectar(self):
        
        try:
            self.connection = mysql.connector.connect(
                host=self.host,
                user=self.user,
                password=self.password,
                database=self.database
            )
            if self.connection.is_connected():
                self.cursor = self.connection.cursor()
                print("Conexión exitosa a la base de datos")
        except Error as e:
            print(f"Error al conectar a la base de datos: {e}")
            return None
        return self.connection
    
    def insertar(self, query, params):
        try:
            self.cursor.execute(query, params)
            self.connection.commit()
            print("Datos insertados correctamente")
        except Error as e:
            print(f"Error al insertar datos: {e}") 

    
    def consulta(self,query,params=None):
        try:
            self.cursor.execute(query,params)if params else self.cursor.execute(query)
            if query.strip().lower().startswith("select"):
                return self.cursor.fetchall()
        except Error as e:
            print(f'Error al ejecutar la consulta {e}')
            return None
        
    def desconectar(self):
        if self.connection.is_connected():
            self.cursor.close()
            self.connection.close()
            print('Conexion cerrada bye')
            
    

In [4]:
conexion = ConexionDB(host="127.0.0.1", user="root", password="21011996", database="inyeccion_produccion")
#querys
conexion.conectar() #CONEXION PARA PRUEBA POSTERIOR 
#Pedir turnos para Entry turnos
query_turnos="""SELECT CONCAT(ID_TURNOS," - ",NOMBRE) as TURNO FROM TURNOS"""
query_maquinas="""SELECT CONCAT (ID_MAQUINAS, " - ",NOMBRE," - ",CAPACIDAD) AS MAQUINA  FROM MAQUINAS"""
query_cabidades_maquina="""SELECT NOMBRE FROM MOLDES WHERE ID_MAQUINA_UTIL=%s"""
query_empleados="""SELECT CONCAT(ID, " - ", NOMBRE) FROM EMPLEADOS"""

print(conexion.consulta(query_empleados,)) #PRUEBA EN LOS QUERYS
conexion.desconectar()

Conexión exitosa a la base de datos
[('1 - DIEGO',), ('2 - BRAYAN',), ('3 - BRANDON',), ('4 - ANTONIO',), ('5 - GILBERTO',), ('6 - MARCO',), ('7 - DANIEL',), ('8 - CORREA',)]
Conexion cerrada bye


In [5]:
def guardar_datos():
    # Recoger los datos de los campos
    nombre_empleado = int(nombre_var.get().strip()[0])  # Nombre del empleado
    fecha_hora = campo_fecha.get().strip()       # Fecha seleccionada
    tiempo=int(tiempo_var.get().strip())
    turno = int(turno_var.get().strip()[0])
    print(turno)
    maquina=int(maquina_var.get().strip()[0])
    inyecciones = campo_inyecciones.get().strip()  # Número de inyecciones
    observaciones = campo_observaciones.get("1.0", tk.END).strip()  # Observaciones
    cabidades = cabidades_var.get().strip()    # Moldes/Cavidades

    # Validar campos obligatorios
    if not nombre_empleado or not fecha_hora or not turno or not inyecciones:
        print("Por favor, llena todos los campos obligatorios")
        return

    try:
        fecha_hora_formateada = datetime.strptime(fecha_hora, "%d/%m/%Y").strftime("%Y-%m-%d %H:%M:%S")
    except ValueError as e:
        print(f"Error en el formato de la fecha: {e}")
        return

    # Consulta SQL ajustada para insertar en la tabla EVENTOS
    query_insertar = """
    INSERT INTO EVENTOS (NOMBRE, FECHA_HORA, TURNO, INYECCIONES, OBSERVACIONES, CABIDADES_MOLDE,MAQUINA,HORAS_TRABAJADAS)
    VALUES (%s, %s, %s, %s, %s, %s,%s,%s)
    """
    parametros = (nombre_empleado, fecha_hora_formateada, turno, inyecciones, observaciones, cabidades,maquina,tiempo)

    # Intentar insertar en la base de datos
    try:
        conexion.insertar(query_insertar, parametros)
        print("Datos guardados correctamente en la base de datos")
    except Exception as e:
        print(f"Error al guardar los datos: {e}")


def obtener_moldes_por_maquina():
    """Devuelve la lista de moldes compatibles con la máquina seleccionada."""
    maquina_seleccionada = maquina_var.get().strip().strip("{}").split('-')[0].strip()  # Obtener solo el ID de la máquina
 
    if maquina_seleccionada:
        try:
            moldes = conexion.consulta(query_cabidades_maquina, (maquina_seleccionada,))
            return [molde[0] for molde in moldes]  # Lista de nombres de moldes 
        except Exception as e:
            print(f"Error al obtener moldes: {e}")
            return []
    return []


def actualizar_moldes(event):
    moldes_disponibles = obtener_moldes_por_maquina()
    cabidades_combo["values"] = moldes_disponibles
    cabidades_var.set("")


def obtener_lista_empleados(lista):
    try:
        
        return [x[0] for x in lista]
    except Exception as e:
        print(f"Error al obtener moldes: {e}")
        return []
    
def autoacompletar_nombres(event):
    entrada=nombre_var.get().lower()
    coincidencias=[nombre[0] for nombre in nombres_empleados if entrada in nombre[0].lower()]
    nombre_combo["values"]=coincidencias
    if coincidencias and entrada:
        nombre_combo.event_generate("<Down>") 


def autocompletar_tiempos(event):
    entrada = tiempo_var.get().lower()
    coincidencias = [tiempo for tiempo in tiempos]
    tiempo_combo["values"] = coincidencias  # Actualiza las opciones
    if coincidencias:
        tiempo_combo.event_generate("<Down>")  # Despliega opciones si hay coincidencias

def enfocar_y_habilitar(event):
    """Permite escribir sin desplegar inmediatamente la lista."""
    nombre_combo["state"] = "normal"  # Habilita la escritura
    nombre_combo.icursor("end")  # Coloca el cursor







In [6]:
tiempos = [i for i in range(0,13)]
tiempos


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [7]:
ventana=tk.Tk()
ventana.title("Produccion inyeccion")
ventana.geometry("750x480")


ventana.grid_columnconfigure(0, weight=0)  # Columna de las etiquetas
ventana.grid_columnconfigure(1, weight=0)


In [8]:

#ETIQUETAS

etiqueta=tk.Label(ventana,text="Registro de eventos")
etiqueta.grid(row=0, column=0, columnspan=7, padx=100, pady=10 , sticky='nsew')

label_fecha=tk.Label(ventana,text="Fecha:")
label_fecha.grid(row=1,column=0,padx=20,pady=5, sticky='e')

label_inicio=tk.Label(ventana,text="Tiempo")
label_inicio.grid(row=1,column=2,padx=20,pady=5, sticky='e')

label_turno=tk.Label(ventana,text="Id turno")
label_turno.grid(row=2,column=0,padx=20,pady=5 , sticky='e')

label_maquina=tk.Label(ventana,text="Nombre Maquina")
label_maquina.grid(row=3,column=0,padx=20,pady=5, sticky='e')

label_nombre = tk.Label(ventana, text="Nombre:")
label_nombre.grid(row=4, column=0, padx=20, pady=5, sticky='e')

label_cabidad=tk.Label(ventana, text="Cabidad/Molde")
label_cabidad.grid(row=5,column=0, padx=20,pady=5, sticky='e')

label_inyecciones=tk.Label(ventana,text="Cantidad Inyecciones")
label_inyecciones.grid(row=6,column=0,padx=20,pady=5, sticky='e')

label_comida=tk.Label(ventana,text="Horario Comida")
label_comida.grid(row=7,column=0,padx=20,pady=5, sticky='e')

label_observaciones=tk.Label(ventana,text="Observaciones")
label_observaciones.grid(row=8,column=0,padx=20,pady=5, sticky='ne')




In [9]:
# CAMPOS DE ENTRADA
campo_fecha = DateEntry(ventana, width=20,background='darckblue', foreground='white',date_pattern='dd/mm/yyyy',)  # Ancho fijo
campo_fecha.grid(row=1, column=1, padx=5, pady=5)


tiempo_var = tk.StringVar()
tiempo_combo = ttk.Combobox(ventana, textvariable=tiempo_var)
tiempo_combo["values"] = tiempos  # Cargar todos los horarios
tiempo_combo.grid(row=1, column=3, padx=10, pady=10)
tiempo_combo.bind("<KeyRelease>", autocompletar_tiempos)

turno_var = tk.StringVar()
conexion.conectar()
turnos=conexion.consulta(query_turnos)
turno_combo = ttk.Combobox(ventana, textvariable=turno_var, values=[turno[0] for turno in turnos])
turno_combo.grid(row=2, column=1, padx=20, pady=5)

maquina_var=tk.StringVar() 
maquinas=conexion.consulta(query_maquinas)
maquina_combo = ttk.Combobox(ventana, textvariable=maquina_var,values=[maquina[0] for maquina in maquinas ])
maquina_combo.grid(row=3, column=1, padx=20, pady=5)
maquina_combo.bind("<<ComboboxSelected>>", actualizar_moldes)

nombre_var=tk.StringVar()
nombres_empleados=conexion.consulta(query_empleados)
nombre_combo=ttk.Combobox(ventana,textvariable=nombre_var,values=[nombre[0] for nombre in nombres_empleados],state="normal")
nombre_combo.grid(row=4, column=1, padx=20, pady=5)
nombre_combo.bind("<KeyRelease>",autoacompletar_nombres)
nombre_combo.bind("<FocusIn>", enfocar_y_habilitar) 

cabidades_var = tk.StringVar()
cabidades_combo = ttk.Combobox(ventana, textvariable=cabidades_var, values=[])
cabidades_combo.grid(row=5, column=1, padx=20, pady=5)


campo_inyecciones = tk.Entry(ventana, width=45)
campo_inyecciones.grid(row=6, column=1, padx=20, pady=5)

campo_comida = tk.Entry(ventana, width=45)
campo_comida.grid(row=7, column=1, padx=20, pady=5)

campo_observaciones = tk.Text(ventana, width=35, height=5)
campo_observaciones.grid(row=8, column=1, padx=20, pady=5)




#ventana.mainloop()


Conexión exitosa a la base de datos


In [10]:
boton_guardar = tk.Button(ventana, text="Guardar", command=guardar_datos)
boton_guardar.grid(row=9, column=1, padx=20, pady=10)

In [11]:
ventana.mainloop()

conexion.desconectar()

1
Datos insertados correctamente
Datos guardados correctamente en la base de datos
1
Datos insertados correctamente
Datos guardados correctamente en la base de datos
1
Datos insertados correctamente
Datos guardados correctamente en la base de datos
1
Datos insertados correctamente
Datos guardados correctamente en la base de datos
1
Datos insertados correctamente
Datos guardados correctamente en la base de datos
1
Datos insertados correctamente
Datos guardados correctamente en la base de datos
1
Datos insertados correctamente
Datos guardados correctamente en la base de datos
2
Datos insertados correctamente
Datos guardados correctamente en la base de datos
2
Datos insertados correctamente
Datos guardados correctamente en la base de datos
1
Datos insertados correctamente
Datos guardados correctamente en la base de datos
1
Datos insertados correctamente
Datos guardados correctamente en la base de datos
2
Datos insertados correctamente
Datos guardados correctamente en la base de datos
2
Da